In [1]:
from data import Data
from dimension_reduction import PCADimensionReduction
from autoencoder import Autoencoder
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import optimizers

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_filepath = "data/by_type/t2/counts_ctc_simulated_123_5k_t2.tsv"
true_results_filepath = "data/by_type/t2/ids_ctc_simulated_123_5k_t2.tsv"
train_indices_filepath = "data/by_type/t2/train_indices.npy"
test_indices_filepath = "data/by_type/t2/test_indices.npy"

SEED = 42
FOLD_NUMBER = 3

# You can change these values to work better for models
CUT_BY_MAX_THRESHOLD = 4

FILTER_BY_ZERO_THRESHOLD = 0.97

In [3]:
data_object = Data(data_filepath, true_results_filepath)

train_data, test_data, train_true_results, test_true_results = data_object.load_train_test_split(train_indices_filepath, test_indices_filepath)

In [4]:
healthy_train_data, cancer_train_data, healthy_test_data, cancer_test_data = data_object.load_train_test_healthy_cancer_data()


In [5]:
def create_autoencoder(input_shape):
    model = Sequential()
    model.add(Dense(50, input_dim=input_shape, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(input_shape))
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

## Autoencoder

### Regular data

In [6]:
autoencoder = create_autoencoder(healthy_train_data.shape[1])
autoencoder.fit(healthy_train_data, healthy_train_data, batch_size=16, epochs=100)

2022-11-27 23:21:19.866083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:21:19.885946: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:21:19.886295: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:21:19.886738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Epoch 1/100
190/190 [==============================] - 1s 993us/step - loss: 0.1640
Epoch 2/100
  1/190 [..............................] - ETA: 0s - loss: 0.1440

2022-11-27 23:21:21.204983: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


190/190 [==============================] - 0s 991us/step - loss: 0.1256
Epoch 3/100
190/190 [==============================] - 0s 992us/step - loss: 0.1250
Epoch 4/100
190/190 [==============================] - 0s 978us/step - loss: 0.1247
Epoch 5/100
190/190 [==============================] - 0s 992us/step - loss: 0.1243
Epoch 6/100
190/190 [==============================] - 0s 988us/step - loss: 0.1230
Epoch 7/100
190/190 [==============================] - 0s 977us/step - loss: 0.1223
Epoch 8/100
190/190 [==============================] - 0s 981us/step - loss: 0.1219
Epoch 9/100
190/190 [==============================] - 0s 987us/step - loss: 0.1215
Epoch 10/100
190/190 [==============================] - 0s 978us/step - loss: 0.1210
Epoch 11/100
190/190 [==============================] - 0s 983us/step - loss: 0.1203
Epoch 12/100
190/190 [==============================] - 0s 980us/step - loss: 0.1197
Epoch 13/100
190/190 [==============================] - 0s 979us/step - loss: 0.1192


In [7]:
autoencoder_object = Autoencoder(autoencoder, healthy_train_data, healthy_test_data, cancer_train_data, cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 704us/step
Healthy train RMSE: 0.3308492991369139
Healthy test RMSE: 0.3351893987732267
Cancer train RMSE: 0.5219696312499756
Cancer test RMSE: 0.515401636769101
Threshold: 0.4264094651934448


In [8]:
autoencoder_object.display_regular_metrics(train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(test_data, test_true_results, "Test")

97/97 [==============================] - 0s 669us/step
Train balanced accuracy: 0.989799276077657
Train f1 score: 0.5303030303030303
Train precision: 0.36082474226804123
Train recall: 1.0
42/42 [==============================] - 0s 680us/step
Test balanced accuracy: 0.9884881043745204
Test f1 score: 0.5
Test precision: 0.3333333333333333
Test recall: 1.0


### Scaled data

In [6]:
scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_train_data, scaled_cancer_test_data = data_object.get_scaled_healthy_train_test_cancer_data()
autoencoder = create_autoencoder(scaled_healthy_train_data.shape[1])
autoencoder.fit(scaled_healthy_train_data, scaled_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 1s 3ms/step - loss: 0.9309
Epoch 2/100
190/190 [==============================] - 1s 3ms/step - loss: 0.9057
Epoch 3/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8967
Epoch 4/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8897
Epoch 5/100
190/190 [==============================] - 0s 3ms/step - loss: 0.8857
Epoch 6/100
190/190 [==============================] - 0s 3ms/step - loss: 0.8823
Epoch 7/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8795
Epoch 8/100
190/190 [==============================] - 1s 4ms/step - loss: 0.8774
Epoch 9/100
190/190 [==============================] - 0s 3ms/step - loss: 0.8757
Epoch 10/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8755
Epoch 11/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8739
Epoch 12/100
190/190 [==============================] - 1s 3ms/step - loss: 0.8715
Epoch 13/100


In [7]:
autoencoder_object = Autoencoder(autoencoder, scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_train_data, scaled_cancer_test_data)
autoencoder_object.display_average_rmse()

Healthy train RMSE: 0.8964106085649064
Healthy test RMSE: 0.9410028734900082
Cancer train RMSE: 7.374979483164166
Cancer test RMSE: 7.049740113746673
Threshold: 4.135695045864536


In [8]:
scaled_train_data = data_object.scaled_data_by_healthy_train(train_data)
scaled_test_data = data_object.scaled_data_by_healthy_train(test_data)

autoencoder_object.display_regular_metrics(scaled_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(scaled_test_data, test_true_results, "Test")

Train balanced accuracy: 1.0
Train f1 score: 1.0
Train precision: 1.0
Train recall: 1.0
Test balanced accuracy: 0.9984650805832693
Test f1 score: 0.8823529411764706
Test precision: 0.7894736842105263
Test recall: 1.0


### Cut by max data

In [12]:
cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_train_data, cut_by_max_cancer_test_data = data_object.get_cut_by_max_healthy_train_test_cancer_data(CUT_BY_MAX_THRESHOLD)
autoencoder = create_autoencoder(cut_by_max_healthy_train_data.shape[1])
autoencoder.fit(cut_by_max_healthy_train_data, cut_by_max_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 958us/step - loss: 1.9081
Epoch 2/100
190/190 [==============================] - 0s 916us/step - loss: 0.5626
Epoch 3/100
190/190 [==============================] - 0s 915us/step - loss: 0.5040
Epoch 4/100
190/190 [==============================] - 0s 930us/step - loss: 0.4524
Epoch 5/100
190/190 [==============================] - 0s 943us/step - loss: 0.4212
Epoch 6/100
190/190 [==============================] - 0s 949us/step - loss: 0.4019
Epoch 7/100
190/190 [==============================] - 0s 929us/step - loss: 0.3886
Epoch 8/100
190/190 [==============================] - 0s 918us/step - loss: 0.3807
Epoch 9/100
190/190 [==============================] - 0s 925us/step - loss: 0.3762
Epoch 10/100
190/190 [==============================] - 0s 928us/step - loss: 0.3733
Epoch 11/100
190/190 [==============================] - 0s 919us/step - loss: 0.3707
Epoch 12/100
190/190 [==============================] - 0s 923us/step - lo

In [13]:
autoencoder_object = Autoencoder(autoencoder, cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_train_data, cut_by_max_cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 585us/step
Healthy train RMSE: 0.5592430466875161
Healthy test RMSE: 0.5724424316230147
Cancer train RMSE: 0.7117419968898846
Cancer test RMSE: 0.6693830839467317
Threshold: 0.6354925217887004


In [14]:
cut_by_max_train_data = data_object.cut_by_max_healthy_data(train_data, CUT_BY_MAX_THRESHOLD)
cut_by_max_test_data = data_object.cut_by_max_healthy_data(test_data, CUT_BY_MAX_THRESHOLD)

autoencoder_object.display_regular_metrics(cut_by_max_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(cut_by_max_test_data, test_true_results, "Test")

97/97 [==============================] - 0s 556us/step
Train balanced accuracy: 0.7052037794387251
Train f1 score: 0.06111111111111111
Train precision: 0.032116788321167884
Train recall: 0.6285714285714286
42/42 [==============================] - 0s 557us/step
Test balanced accuracy: 0.7074699411614224
Test f1 score: 0.05665722379603398
Test precision: 0.029585798816568046
Test recall: 0.6666666666666666


### Filtered by zero values for scaled data

In [9]:
filtered_by_zero_values_train_data, filtered_by_zero_values_test_data, filtered_by_zero_healthy_train_data, filtered_by_zero_healthy_test_data, cancer_train_data, cancer_test_data = data_object.get_filtered_by_zero_data(scaled_train_data, scaled_test_data, FILTER_BY_ZERO_THRESHOLD)

Number of deleted columns: 93


In [10]:
autoencoder = create_autoencoder(filtered_by_zero_healthy_train_data.shape[1])
autoencoder.fit(filtered_by_zero_healthy_train_data, filtered_by_zero_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 1s 3ms/step - loss: 0.9807
Epoch 2/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9569
Epoch 3/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9461
Epoch 4/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9410
Epoch 5/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9379
Epoch 6/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9354
Epoch 7/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9335
Epoch 8/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9318
Epoch 9/100
190/190 [==============================] - 0s 3ms/step - loss: 0.9292
Epoch 10/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9269
Epoch 11/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9254
Epoch 12/100
190/190 [==============================] - 0s 2ms/step - loss: 0.9242
Epoch 13/100


In [11]:
autoencoder_object = Autoencoder(autoencoder, filtered_by_zero_healthy_train_data, filtered_by_zero_healthy_test_data, cancer_train_data, cancer_test_data)
autoencoder_object.display_average_rmse()

Healthy train RMSE: 0.9208914492116463
Healthy test RMSE: 0.9648759938904269
Cancer train RMSE: 7.534162684916948
Cancer test RMSE: 7.228558513737431
Threshold: 4.2275270670642975


In [12]:
autoencoder_object.display_regular_metrics(filtered_by_zero_values_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(filtered_by_zero_values_test_data, test_true_results, "Test")

Train balanced accuracy: 1.0
Train f1 score: 1.0
Train precision: 1.0
Train recall: 1.0
Test balanced accuracy: 0.9984650805832693
Test f1 score: 0.8823529411764706
Test precision: 0.7894736842105263
Test recall: 1.0
